Julia's `broadcast` framework allows programmers to syntactically fuse pointwise array operations using a `.` after function application to indicate that the function should be applied pointwise to the elements of it's arguments. As an example, let's take the pointwise product of three arrays:

In [1]:
A = [ 1  0  1  0  ]

B = [ 1  2  3  4  ;
      5  6  7  8  ;
      9  10 11 12 ;
      13 14 15 16 ]

C = [ 1;
     -1;
      1;
     -1]

A.*B.*C

4×4 Array{Int64,2}:
   1  0    3  0
  -5  0   -7  0
   9  0   11  0
 -13  0  -15  0

Each of our arrays has a finite set of dimensions (`1x4`, `4x4`, and `4`, to be precise), but from the point of view of broadcast, each array also has an additional, infinite number of implicit dimensions of size `1` (`1x4x1x1x...`, `4x4x1x1x...`, and `4x1x1x1x...`). To take the pointwise product, dimensions of size `1` are expanded to match larger dimensions. If we were to put each dimension on a line, we might get a picture that looks something like:

```
...               ...               ...               ...

 3  ------------- (1) ------------- (1) ------------- (1)

 2  -------------  4  -------------  4  ------------- (1)

 1  -------------  1  -------------  4  -------------  4

dim                A       .*        B       .*        C
```

I've taken the liberty of parenthetizing the implicit dimensions. The expanded view of each array would look something like:

```
                  (A)      .*        B       .*       (C)

            [ 1  0  1  0  ;   [ 1  2  3  4  ;   [ 1  1  1  1  ;
              1  0  1  0  ;     5  6  7  8  ;    -1 -1 -1 -1  ;
              1  0  1  0  ;     9  10 11 12 ;     1  1  1  1  ;
              1  0  1  0  ]     13 14 15 16 ]    -1 -1 -1 -1  ]
```

Note that `broadcast` returns an array with a number of dimensions equal to the highest "non-implicit" dimension, so that we don't get arrays with infinite dimensions as output.

While `broadcast` allows us to expand dimensions of size `1`, it does not provide any way to squish or swap dimensions. We introduce the `swizzle` operation to remedy the situation. `swizzle` sends the dimensions of its argument to some output dimensions, optionally reducing along dimensions with a specified reduction operator, according to a mask. The mask is an iterable object with eltypes of `Int` and `Drop`, where `drop` is an instance of the singleton type `Drop` that specifies a dimension is to be reduced over. If a mask is too short, implicit `drop` elements are added.

As a first example, let's swizzle `A` into a column vector with the mask `(drop, 1)`. The operation can be visualized as:

```
...               ...               ...               ...

 3  ------------- (1) ------------- drop              (1)

 2  -------------  4  -------------  1  -----\        (1)
                                              \
 1  -------------  1  ------------- drop       \-----  4

dim                A                mask     swizzle(A, (drop, 1))
```

In [2]:
using Swizzle
swizzle(A, (drop, 1))

4-element Array{Int64,1}:
 1
 0
 1
 0

We can use swizzles to quickly create arrays with many dimensions. Like `broadcast`, `swizzle` will produce an output array just large enough to hold all of the "non-implicit" dimensions from its argument. Consider swizzling `A` with respect to the mask `(1, 3, 4)`:
```
...               ...               ...               ...

 4  ------------- (1) ------------- drop       /----- (1)
                                              /
 3  ------------- (1) -------------  4  -----/ /-----  4
                                              /
 2  -------------  4  -------------  3  -----/         1

 1  -------------  1  -------------  1  -------------  1

dim                A                mask     swizzle(A, (1, 3, 4))
```

In [3]:
swizzle(A, (1, 3, 4))

1×1×4 Array{Int64,3}:
[:, :, 1] =
 1

[:, :, 2] =
 0

[:, :, 3] =
 1

[:, :, 4] =
 0

My favorite feature of swizzles is that they can express reductions! For example, we can take the sum of `A` by swizzling with the mask `(drop, drop)` and the operation `+`:
```
...               ...               ...               ...

 3  ------------- (1) ------------- drop              (1)

 2  -------------  4  ------------- drop              (1)

 1  ------------- (1) ------------- drop              (1)

dim                A                mask    swizzle(A, (drop, drop), +)
```

In [4]:
swizzle(A, (drop, drop), +)

0-dimensional Array{Int64,0}:
2

The `swizzle` operation can be fused with broadcast operations using the `Swizzle(mask, op)` type, which produces a lazy swizzle of its argument when it is broadcasted as a function. It can be convenient to define shorthand constructors for `Swizzle` types. `Sum(dims...)` produces a `Swizzle` which sums the dimensions given in `dims` and shifts remaining dimensions down. `Beam(dims...)` is a shorthand for `Swizzle(dims, nooperation)`. We can now express an entire matrix multiply operation using `broadcast` and `swizzle` operations! Remember that for matrices `D` and `E`, if `F == D * E` then $$F_{ij} = \sum_k D_{ik} * E_{kj}$$ With swizzles, we express this as: ```Sum(2).(Beam(1, 2).(D).*Beam(2, 3).(E))```

In [5]:
D = rand(5,7)
E = rand(7,6)
Sum(2).(Beam(1, 2).(D).*Beam(2, 3).(E)) ≈ D*E


true

Let's break down what just happened:
```
...               ...               ...               ...               ...
                                                                \
 4  ------------- (1) ------------- (1) -------------  3  -----\ \----- (1)
                                                                \
 3  ------------- (1) -------------  6  -------------  2  -----\ \----- (1)
                                                                \
 2  -------------  7  -------------  7  ------------- drop       \-----  6

 1  -------------  5  -------------  1  -------------  1  -------------  5

dim          Beam(1, 2).(D) .* Beam(2, 3).(E)       Sum(2).            D * E
```
Before the `Sum(2)` node, we have the expression `Beam(1, 2).(D).*Beam(2, 3).(E)`. `Beam(1, 2).(D)` is just `D`, and `Beam(2, 3).(E)` is just `E` shifted up one dimension, so `(Beam(1, 2).(D).*Beam(2, 3).(E))[i, k, j]` is $D_{ik} * E_{kj}$. At this point, we need only to sum over $k$, so we use `Sum(2)`, producing the matrix product.